In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.ensemble import BaggingRegressor

import numba
import joblib

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas_datareader.data as DataReader
import yfinance as yfin
yfin.pdr_override()

import json
import requests
import urllib.request

import yaml
import configparser
from pywinauto import Desktop, Application, findwindows
from pywinauto.keyboard import SendKeys, send_keys
import time

from datetime import datetime as dt
import os

from IPython.display import clear_output

In [2]:
# 特徴量
FEATURES = ['pre_o2c', 'pre_c2o', 'pre_o2c2', 'pre_c2o2', 'pre_o2c3', 'pre_c2o3', 'dow_o2c']

# 実行環境
ENV = 'prod' #prod or val

# 関数定義

## kabu-station操作

In [26]:
# kabu-station起動
class KabuStationApi:
    def __init__(self):
        # config
        api = {}
        config_ini = configparser.ConfigParser()
        config_ini.read('..//config.ini', encoding='utf-8')
        if ENV=='prod':
            api['number'] = config_ini.get('au', 'number_prod')
            api['pswrd'] = config_ini.get('au', 'pswrd_prod')
        else:
            api['number'] = config_ini.get('au', 'number_val')
            api['pswrd'] = config_ini.get('au', 'pswrd_val')

        # IE経由で株ステーションを起動
        app = Application().start("C:\\Program Files\Internet Explorer\iexplore.exe http://download.r10.kabu.co.jp/kabustation/KabuStation.application")
        time.sleep(20)

        # 起動したウィンドウにフォーカスを当てる
        app = app.connect(title_re="ログイン")
        btn = app['ログイン']
        btn.set_focus()
        time.sleep(10)

        # パスワードの入力＆Enter
        send_keys(api['pswrd'])
        time.sleep(5)
        send_keys("{ENTER}")
        time.sleep(5)

        # 初期化＆トークン取得
        url = 'http://localhost:'+api['number']+'/kabusapi/token'
        headers = {'content-type': 'application/json'}
        payload = json.dumps(
            {'APIPassword': api['pswrd'],}
        ).encode('utf8')
        response = requests.post(url, data=payload, headers=headers)

        api['token'] = json.loads(response.text)['Token']
        api['app'] = app
        
        self.api = api
        # return api#app, json.loads(response.text)['Token']

    # kabu-station終了
    def __del__(self):
        self.api['app'].kill()

    # 新規注文
    def create_order(symbol=None, qty=None, side="1", price=None, frontOrderType=21, 
                     cashMargin=2, marginTradeType=1, delivType=0, closePositionOrder=None):
        url = 'http://localhost:'+self.api['number']+'/kabusapi/sendorder'
        headers = {'content-type': 'application/json', 'X-API-KEY': self.api['token']}
        params = {
              "Password": api['pswrd'],
              "Symbol": symbol,
              "Exchange": 1,#1: 東証
              "SecurityType": 1,#1: 株式
              "Side": str(side),#1: 売り, #2: 買い
              "CashMargin": cashMargin,#1: 現物, 2: 新規, 3: 返済
              "MarginTradeType": marginTradeType,#1 制度信用, 2 一般信用（長期）, 3: 一般信用（デイトレ）
              "DelivType": delivType, #0: 指定なし, 1: 自動振替, 2: お預り金
                #※現物買は指定必須。
                #※現物売は「0(指定なし)」を設定
                #※信用新規は「0(指定なし)」を設定
                #※信用返済は指定必須
              "AccountType": 4,#2: 一般, 4: 特定, 12: 法人,
              "Qty": qty,#注文数量
              "FrontOrderType": frontOrderType,#21 or 16
                #10: 成行->0, 13: 寄成（前場）->0, 14: 寄成（後場）->0, 15: 引成（前場）->0, 16-: 引成（後場）->0
                #17: IOC成行->0, 20: 指値->発注したい金額, 21: 寄指（前場）->発注したい金額, 22: 寄指（後場）-> 発注したい金額
                #23: 引指（前場）->発注したい金額, 24: 引指（後場）->発注したい金額, 25: 不成（前場）->発注したい金額
                #26: 不成（後場）->発注したい金額, 27: IOC指値->発注したい金額, 30: 逆指値->発注したい金額
              "Price": price,
              "ExpireDay": 0,#yyyyMMdd形式。「0」を指定すると、kabuステーション上の発注画面の「本日」に対応する日付として扱います。
            }
        if closePositionOrder is not None:
            params['ClosePositionOrder'] = closePositionOrder
        payload = json.dumps(
            params
        ).encode('utf8')

        response = requests.post(url, data=payload, headers=headers)
        res = json.loads(response.text)
        return res

    # 有効注文の確認
    def check_orders(self):
        url = 'http://localhost:'+self.api['number']+'/kabusapi/orders'
        response = requests.get(url, headers={'X-API-KEY': self.api['token'],})
        orders = json.loads(response.text)

        return orders

    # 建玉確認
    def check_positions(self):
        url = 'http://localhost:'+self.api['number']+'/kabusapi/positions'
        response = requests.get(url, headers={'X-API-KEY': self.api['token'],})
        positions = json.loads(response.text)
        return positions

    # 有効注文のキャンセル
    def cancel_order(self, oid):
        url = 'http://localhost:'+self.api['number']+'/kabusapi/cancelorder'
        headers = {'content-type': 'application/json', 'X-API-KEY': self.api['token']}
        payload = json.dumps(
            {
                "OrderId": oid,
                "Password": self.api['pswrd']   
            }
        ).encode('utf8')

        response = requests.put(url, data=payload, headers=headers)
        res = json.loads(response.text)
        return res

    # 取引余力
    def get_stock_account_wallet(self):
        url = 'http://localhost:'+self.api['number']+'/kabusapi/wallet/cash'
        response = requests.get(url, headers={'X-API-KEY': self.api['token'],})
        cash = json.loads(response.text)
        cash = cash['StockAccountWallet']
        return cash

    # 銘柄情報の取得
    def get_stock_info(self, symbol):
        url = 'http://localhost:18080/kabusapi/symbol/'+str(symbol)+'@1'
        params = { 'addinfo': 'false' } # true:追加情報を出力する、false:追加情報を出力しない　※追加情報は、「時価総額」、「発行済み株式数」、「決算期日」、「清算値」を意味します
        req = urllib.request.Request('{}?{}'.format(url, urllib.parse.urlencode(params)), method='GET')
        req.add_header('Content-Type', 'application/json')
        req.add_header('X-API-KEY', self.api['token'])
        with urllib.request.urlopen(req) as res:
            content = json.loads(res.read())
        return content

    # 呼び値グループ・値段水準と呼び値の対応
    def calc_pips(self, code, price):
        '''
        https://kabucom.github.io/kabusapi/reference/index.html#operation/symbolGet
        '''
        if code=='10000':#TOPIX100以外
            if price<=3000:
                return 1
            elif price<=5000:
                return 5
            elif price<=30000:
                return 10
            elif price<=50000:
                return 50
            elif price<=300000:
                return 100
            elif price<=500000:
                return 500
            elif price<=3000000:
                return 1000
            elif price<=5000000:
                return 5000
            elif price<=30000000:
                return 10000
            elif price<=50000000:
                return 50000
            else:
                return 100000
        elif code=='10003':#TOPIX100
            if price<=1000:
                return 0.1
            elif price<=3000:
                return 0.5
            elif price<=10000:
                return 1
            elif price<=30000:
                return 5
            elif price<=100000:
                return 10
            elif price<=300000:
                return 50
            elif price<=1000000:
                return 100
            elif price<=3000000:
                return 500
            elif price<=10000000:
                return 1000
            elif price<=30000000:
                return 5000
            else:
                return 10000
        else:
            raise Exception

In [29]:
api = KabuStationApi()
print(api.get_stock_info(symbol=1320))
del api

{'BisCategory': '9999      ', 'TotalMarketValue': None, 'TotalStocks': None, 'FiscalYearEndBasic': None, 'KCMarginBuy': True, 'KCMarginSell': True, 'MarginBuy': True, 'MarginSell': True, 'DisplayName': '大和日経平均/ETF', 'Exchange': 1, 'ExchangeName': '東証ETF/ETN', 'TradingUnit': 1.0, 'PriceRangeGroup': '10003', 'UpperLimit': 32360.0, 'LowerLimit': 22360.0, 'Symbol': '1320', 'SymbolName': 'ダイワ上場投信－日経２２５'}


## pre-order

In [30]:
# data load
def load_data(start="2021-01-01"):
    ## topix2000 etf
    df = DataReader.get_data_yahoo('1320.T', start=start)
    df = df.add_suffix('_topix')

    ## dow
    df_dow = DataReader.get_data_yahoo('^DJI', start=start)
    df_dow = df_dow.add_suffix('_dow')

    ## マージ
    df = pd.merge(
        left=df.reset_index(), right=df_dow.reset_index(), how='left'
    ).fillna(method='ffill').set_index('Date').dropna()
    return df

# 特徴量作成&予測実行
def pred_score(df):
    df = df.assign(
        o2c = lambda df: df['Close_topix']/df['Open_topix']-1,
        
        c2o = lambda df: df['Open_topix']/df['Close_topix'].shift(1)-1,
        dow_o2c = lambda df: df['Close_dow'].shift(1)/df['Open_dow'].shift(1)-1,

        pre_o2c = lambda df: df.o2c.shift(1),
        pre_c2o = lambda df: df['Open_topix'].shift(1)/df['Open_topix'].shift(2)-1,

        pre_o2c2 = lambda df: df.pre_o2c.shift(1),
        pre_c2o2 = lambda df: df.pre_c2o.shift(1),

        pre_o2c3 = lambda df: df.pre_o2c.shift(1),
        pre_c2o3 = lambda df: df.pre_c2o.shift(1),
    )
    
    # model load
    model_o2c = joblib.load('H:\マイドライブ\Jupyter\T2000\model_o2c.xz')
    
    X = df[FEATURES].fillna(0).values
    y = df.o2c.fillna(0).values
    pred = model_o2c.predict(X)

    df = df.assign(
        pred1 = pred,
        side = lambda df: df.pred1.apply(lambda x: 1 if x>0.000 else -1 if  x<-0.000 else 0),
        price_sell = lambda df: df['Close_topix']*0.999,
        price_buy = lambda df: df['Close_topix']*1.001,
        exec1 = lambda df: ((df.price_sell<=df['Open_topix'])&(df.pred1<=-0.000))|((df.price_buy>=df['Open_topix'])&(df.pred1>0.000)),
        rate1 = lambda df: (df.o2c * df.pred1.apply(lambda x: 1 if x>0.000 else -1 if  x<-0.000 else 0)).apply(lambda x: 0 if x==0 else x-0.04/365) * df.exec1,
    )
    
    return df

# 注文情報の作成
def make_preorder(df, api):
    # 銘柄情報の取得
    res = get_stock_info(api, 1320)
    pips = calc_pips(res['PriceRangeGroup'], int(df.Close_topix.iloc[-1]))
    
    preorder = {
        "symbol":1320,
        "qty":int(res['TradingUnit']),
        "side":"1" if df.side.iloc[-1]<0 else "2",
        "price":int(df.price_sell.iloc[-1]//pips)*pips if df.side.iloc[-1]<0 else int(df.price_buy.iloc[-1]//pips)*pips,
        "frontOrderType":21,
        "cashMargin":2, 
        "marginTradeType":1, 
        "delivType":0
    }
    return preorder

# 指定時間まで接続を維持しながら待機
def wait(str_time, connection=True):
    target = pd.to_datetime("2030-12-01 "+str_time)
    while True:
        # 30分に1回残高確認
        if connection:
            cash = get_stock_account_wallet(api)
            print(cash)
            os.system('w32tm /resync')

        now = dt.now()
        print(now)
        waitTime = (target-now).seconds%(60*60*24)
        
        if waitTime<30*60:
            time.sleep(waitTime)
            return
        else:
            time.sleep(30*60)

# main

In [ ]:
while(True):
    wait("08:50:00", False)
    #########################################################  AM8:40  #############################################################
    clear_output
    # # 株ステーション起動・api初期化
    api = start_kabu_station()

    # 注文情報の作成
    df = load_data(start="2021-01-01")
    display(df)

    df = pred_score(df)
    display(df)

    preorder = make_preorder(df, api)
    display(preorder)

    # 注文実行
    res = create_order(api=api, symbol=preorder['symbol'], qty=preorder['qty'], 
                       # side=preorder["side"], price=0, frontOrderType=16,
                       side=preorder["side"], price=preorder['price'], frontOrderType=preorder['frontOrderType'],
                      cashMargin=preorder["cashMargin"], marginTradeType=preorder['marginTradeType'], delivType=preorder["delivType"])
    display(res)



    wait("09:10:00")
    #########################################################  AM9:10  #############################################################
    # 残存注文のキャンセル
    orders = check_orders(api)
    for order in orders:
        res = cancel_order(api, order['ID'])
        print(res)

    orders = check_orders(api)
    print(orders)

    # 決済注文
    positions = check_positions(api)
    for pos in positions:
        if pos['Symbol']=='1320':
            side = "2" if  pos['Side']=='1' else "1"#反対売買
            qty = int(pos["LeavesQty"])

            res = create_order(api=api, symbol=1320, qty=qty, side=side, price=0, frontOrderType=16,#引成（後場）
                               cashMargin=3,# 返済
                               closePositionOrder=1,# 古い順・損益低い順に決済
                               marginTradeType=1,# 制度信用
                               delivType=2)# お預かり金
            print(res)

    # kabu-stationの終了
    close_kabu_station(api)
    # break

2022-04-14 08:59:51.260165
2022-04-14 09:29:50.226309
2022-04-14 09:59:50.234976
2022-04-14 10:29:50.237346
2022-04-14 10:59:50.249968
2022-04-14 11:29:50.251146
2022-04-14 11:59:50.259179
2022-04-14 12:29:50.269294
2022-04-14 12:59:50.283312
2022-04-14 13:29:50.291747
2022-04-14 13:59:50.293220
2022-04-14 14:29:50.304085
2022-04-14 14:59:50.304770
2022-04-14 15:29:50.319030
2022-04-14 15:59:50.328711
2022-04-14 16:29:50.329746
2022-04-14 16:59:50.335112
2022-04-14 17:29:50.338167
2022-04-14 17:59:50.340430
2022-04-14 18:29:50.341320
2022-04-14 18:59:50.341703
2022-04-14 19:29:50.355989
2022-04-14 19:59:50.359131
2022-04-14 20:29:50.363632
2022-04-14 20:59:50.375100
2022-04-14 21:29:50.381323
2022-04-14 21:59:50.395414
2022-04-14 22:29:50.398211
2022-04-14 22:59:50.401935
2022-04-14 23:29:50.415992
2022-04-14 23:59:50.427095
2022-04-15 00:29:50.439822
2022-04-15 00:59:50.449408
2022-04-15 01:29:50.456939
2022-04-15 01:59:50.466973
2022-04-15 02:29:50.476057
2022-04-15 02:59:50.483875
2

,Open_topix,High_topix,Low_topix,Close_topix,Adj Close_topix,Volume_topix,Open_dow,High_dow,Low_dow,Close_dow,Adj Close_dow,Volume_dow
Date,,,,,,,,,,,,
2021-01-04,28400.0,28410.0,27810.0,28060.0,28060.0,40006,30627.470703,30674.279297,29881.820312,30223.890625,30223.890625,475080000.0
2021-01-05,27910.0,28060.0,27840.0,27920.0,27920.0,32918,30204.250000,30504.890625,30141.779297,30391.599609,30391.599609,350910000.0
2021-01-06,27860.0,27960.0,27780.0,27820.0,27820.0,33561,30362.779297,31022.650391,30313.070312,30829.400391,30829.400391,500430000.0
2021-01-07,28170.0,28400.0,28160.0,28250.0,28250.0,31251,30901.179688,31193.400391,30897.859375,31041.130859,31041.130859,427810000.0
2021-01-08,28520.0,28940.0,28480.0,28940.0,28940.0,184993,31069.580078,31140.669922,30793.269531,31097.970703,31097.970703,381150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-08,28220.0,28275.0,27825.0,28055.0,28055.0,33618,34569.238281,34908.460938,34470.191406,34721.121094,34721.121094,301750000.0
2022-04-11,27945.0,28075.0,27780.0,27895.0,27895.0,56757,34630.269531,34701.339844,34272.289062,34308.078125,34308.078125,333290000.0
2022-04-12,27625.0,27740.0,27345.0,27450.0,27450.0,90255,34412.511719,34669.968750,34102.808594,34220.359375,34220.359375,333510000.0


,Open_topix,High_topix,Low_topix,Close_topix,Adj Close_topix,Volume_topix,Open_dow,High_dow,Low_dow,Close_dow,...,pre_o2c2,pre_c2o2,pre_o2c3,pre_c2o3,pred1,side,price_sell,price_buy,exec1,rate1
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,28400.0,28410.0,27810.0,28060.0,28060.0,40006,30627.470703,30674.279297,29881.820312,30223.890625,...,NaN,NaN,NaN,NaN,-0.000382,-1,28031.940,28088.060,True,0.011862
2021-01-05,27910.0,28060.0,27840.0,27920.0,27920.0,32918,30204.250000,30504.890625,30141.779297,30391.599609,...,NaN,NaN,NaN,NaN,0.000094,1,27892.080,27947.920,True,0.000249
2021-01-06,27860.0,27960.0,27780.0,27820.0,27820.0,33561,30362.779297,31022.650391,30313.070312,30829.400391,...,-0.011972,NaN,-0.011972,NaN,-0.000732,-1,27792.180,27847.820,True,0.001326
2021-01-07,28170.0,28400.0,28160.0,28250.0,28250.0,31251,30901.179688,31193.400391,30897.859375,31041.130859,...,0.000358,-0.017254,0.000358,-0.017254,-0.000754,-1,28221.750,28278.250,False,-0.000000
2021-01-08,28520.0,28940.0,28480.0,28940.0,28940.0,184993,31069.580078,31140.669922,30793.269531,31097.970703,...,-0.001436,-0.001791,-0.001436,-0.001791,-0.000308,-1,28911.060,28968.940,False,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-08,28220.0,28275.0,27825.0,28055.0,28055.0,33618,34569.238281,34908.460938,34470.191406,34721.121094,...,-0.004549,-0.016351,-0.004549,-0.016351,-0.000800,-1,28026.945,28083.055,True,0.005737
2022-04-11,27945.0,28075.0,27780.0,27895.0,27895.0,56757,34630.269531,34701.339844,34272.289062,34308.078125,...,-0.002140,-0.018898,-0.002140,-0.018898,-0.000287,-1,27867.105,27922.895,True,0.001680
2022-04-12,27625.0,27740.0,27345.0,27450.0,27450.0,90255,34412.511719,34669.968750,34102.808594,34220.359375,...,-0.005847,0.006599,-0.005847,0.006599,-0.000278,-1,27422.550,27477.450,True,0.006225


{'symbol': 1320,
 'qty': 1,
 'side': '1',
 'price': 28210,
 'frontOrderType': 21,
 'cashMargin': 2,
 'marginTradeType': 1,
 'delivType': 0}

{'Result': 0, 'OrderId': '20220415A02N78844175'}

437428.0
2022-04-15 08:50:47.525033
{'Code': 41, 'Message': '該当注文が見つかりません'}
{'Code': 41, 'Message': '該当注文が見つかりません'}
{'Code': 42, 'Message': '該当注文が既に有効期限切れです'}
[{'CashMargin': 2, 'MarginTradeType': 1, 'MarginPremium': None, 'ID': '20220413A02N76379230', 'State': 5, 'OrderState': 5, 'OrdType': 2, 'RecvTime': '2022-04-13T08:50:40.316897+09:00', 'Symbol': '1320', 'SymbolName': 'ダイワ上場投信－日経２２５', 'Exchange': 1, 'ExchangeName': '東証ETF/ETN', 'Price': 27420.0, 'OrderQty': 1.0, 'CumQty': 1.0, 'Side': '1', 'AccountType': 4, 'DelivType': 0, 'ExpireDay': 20220413, 'Details': [{'SeqNum': 1, 'ID': '20220413A02N76379230', 'RecType': 1, 'ExchangeID': None, 'State': 3, 'TransactTime': '2022-04-13T08:50:40.316897+09:00', 'OrdType': 2, 'Price': 27420.0, 'Qty': 1.0, 'ExecutionID': None, 'ExecutionDay': None, 'DelivDay': 20220415, 'Commission': 0.0, 'CommissionTax': 0.0}, {'SeqNum': 4, 'ID': '20220413B02N76379231', 'RecType': 4, 'ExchangeID': '1F111500004043', 'State': 3, 'TransactTime': '2022-04-13T08:50:40

In [40]:
close_kabu_station(api)